In [1]:
#Entropy

import numpy as np

In [2]:
Y = np.array([1,0,0,1,0,1,0,1])

In [11]:
def entropy(var):
    N= var.shape[0]
    values, counts = np.unique(var, return_counts=True)
    
    ent = 0.0
    
    for i in counts:
        p = i/N
        ent += (p * np.log2(p))
        
    return -ent

In [10]:
entropy(Y)

1.0

In [12]:
import pandas as pd

In [14]:
df = pd.read_csv('golf.csv')

In [15]:
df

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [40]:
def divide_data(data, feature):
    #here we are working with data freames
    
    DATA = {}
    
    feat_values = list(data[feature].value_counts().index)
    occurence = list(data[feature].value_counts())
    
    for val in feat_values:
        DATA[val] = {'data' : pd.DataFrame([], columns = data.columns), 'len': 0}
        
    for ix in range (data.shape[0]):
        val = data[feature].iloc[ix]
        DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
        
        idx = feat_values.index(val)
        DATA[val]['len'] = occurence[idx]
    
    return DATA
        

In [41]:
divide_data(df, 'Windy')

/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

{False: {'data':      Outlook Temperature Humidity  Windy Play
  0      sunny         hot     high  False   no
  2   overcast         hot     high  False  yes
  3      rainy        mild     high  False  yes
  4      rainy        cool   normal  False  yes
  7      sunny        mild     high  False   no
  8      sunny        cool   normal  False  yes
  9      rainy        mild   normal  False  yes
  12  overcast         hot   normal  False  yes,
  'len': 8},
 True: {'data':      Outlook Temperature Humidity Windy Play
  1      sunny         hot     high  True   no
  5      rainy        cool   normal  True   no
  6   overcast        cool   normal  True  yes
  10     sunny        mild   normal  True  yes
  11  overcast        mild     high  True  yes
  13     rainy        mild     high  True   no,
  'len': 6}}

In [36]:
#Information Gain

    

In [42]:
def information_gain(data, feature):
    examples = data.shape[0]
    
    DATA = divide_data(data, feature)
    keys = DATA.keys()
    
    ent_of_children = 0.0
    
    for key in keys:
        ent_of_children += ( (DATA[key]['len']/examples) * entropy(DATA[key]['data']['Play']) )
        
    info_gain = entropy(data['Play']) - ent_of_children
    return info_gain

In [43]:
information_gain(df, 'Outlook')

/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

0.24674981977443933

In [45]:
#Building a Decision Tree

In [64]:
class DecisionTree:
    
    #constructor
    def __init__(self, depth=0, max_depth=5):
        #creating a Node
        self.children = {}
        self.fkey = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
    
    def train(self,data):
        features = ['Outlook', 'Temperature', 'Humidity', 'Windy']
        
        info_gain = [ ]
        
        for f in features:
            i_gain = information_gain(data,f)
            info_gain.append(i_gain)
            
        self.fkey = features[np.argmax(info_gain)]
        
        #Splitting the data
        DATA = divide_data(data, self.fkey)
        
        
        #Giving a target label to theNode 
        
        labels = list(data['Play'].value_counts().index)
        
        freq = list(data['Play'].value_counts().values)
        
        self.target = labels[np.argmax(freq)]
        
        
        ##STOPPING CONDITION
        have_data = 0
        keys = DATA.keys()
        
        for key in keys:
            if DATA[key]['len']>0:
                have_data += 1
        
        
        # 1. If it is a pure node
        if have_data < 2:
            return
        
        # 2. you have reached max depth
        if(self.depth >= self.max_depth):
            return
        
        print("\t"*self.depth + "Making tree with - ", self.fkey)
        
        #Recursively Train the child node
        
        for key in keys:
            new_data = DATA[key]['data']
            self.children[key] = DecisionTree(depth = self.depth + 1)
            self.children[key].train(new_data)
            
        return
    
    def predict(self, test):
        if self.children == {}:
            return self.target
        
        return self.children[test[self.fkey][0]].predict(test)

In [65]:
#Explore the model

In [66]:
model = DecisionTree()

In [67]:
model.train(df)

/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

Making tree with -  Outlook


/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

	Making tree with -  Humidity


/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

	Making tree with -  Windy


/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DATA[val]['data'] = DATA[val]['data'].append(data.iloc[ix])
/var/folders/1d/z37fr91n0wv10sbrhgdnfzjh0000gp/T/ipykernel_69442/93950325.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

In [70]:
x_test = pd.DataFrame([['sunny', 'hot', 'high', False]], columns=list(df.columns.values[:-1]))

In [71]:
model.predict(x_test)

'no'